In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from preprocess import mediapipe_detection,landmarks

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def draw_styled_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (121,22,76), thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (121,44,250), thickness = 2, circle_radius = 2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (245,117,66), thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (245,66,230), thickness = 2, circle_radius = 2))

def extract_keypoints(results):
    left_hand = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark] ).flatten() if results.left_hand_landmarks else np.zeros(63)
    right_hand = np.array(([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark] )).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([left_hand,right_hand])

In [11]:
import pickle
from math import ceil,floor
Pkl_Filename = "Hand_ML_Model.pkl"
with open(Pkl_Filename, 'rb') as file:  
    rf = pickle.load(file)

rf

RandomForestRegressor(n_estimators=1000, random_state=28)

In [31]:
tt = {0:'A',1:'B',2:'C',3:'D',4:'E',
     5:'F',6:'G',7:'H',8:'I',9:'J',
     10:'K',11:'L',12:'M',13:'N',14:'O',
     15:'P',16:'Q',17:'R',18:'Reset',19:'S',
     20:'Space',21:'T',22:'U',23:'V',24:'W',
     25:'X',26:'Y',27:'Z'}

def pred(test_path):
    x = rf.predict(test_path.reshape(1,126))
    #op = floor(x[0])
    if int(str(x[0]).split('.')[1][:2])>80:
        op = ceil(x[0])
    else:
        op = floor(x[0])
    return tt[op]

In [32]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.4) as holistic:
    
    while True: 
        
        _,frame = cap.read()
        image,results = mediapipe_detection(frame,holistic)
        draw_styled_landmarks(image,results)
        

        #cv2.imshow("sign_videos",image)

        keypoints = extract_keypoints(results)
        max_frame = []
        pp = pred(keypoints)
        cv2.putText(image,'Image for{}'.format(pp),(15,12),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,255),1,cv2.LINE_AA)
        cv2.imshow("sign_videos",image)
        '''
        for xx in range(0,20):
            pp = max_frame.append(pred(keypoints))
        else:
            print(max(pp))

        continue

        '''
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()